# Order By

Now you'll learn how to change the order of your results using the **ORDER BY** clause, and you'll explore a popular use case by applying ordering to dates. To illulstrate what you'll learn in this tutorial, we'll work with a slighly modified version of our familiar `pets` table.

![alt text](pets2.PNG "pets")

## ORDER BY

**ORDER BY** is usually the last clause in your query, and it sorts the results returned by the rest of your query.

Notice that the rows are not ordered by the `ID` column. We can quickly remedy this with the query below.

    SELECT ID, Name, Animal
    FROM `bigquery.public-data.pet_records.pets`
    ORDER BY ID
    
The **ORDER BY** clause also works for columns containing text, where the results show up in alphabetical order.

    SELECT ID, Name, Animal
    FROM `bigquery.public-data.pet_records.pets`
    ORDER BY Animal
    
You can reverse the order using the **DESC** argument (short for 'descending'). The next query sorts the table by the `Animal` column, where the values that are last in alphabetical order are returned fist.

    SELECT ID, Name, Animal
    FROM `bigquery.public-data.pet_records.pets`
    ORDER BY Animal DESC
    
## Dates

Next, we'll talk about dates, becase they come up very frequently in real-world databases. There are two ways that dates can be stored in BigQuery: as DATE or as DATETIME.

The **DATE** format has the year first, then the month and then the day. It looks like this:

    YYYY-[M]M-[D]D
    
    - YYYY: Four-digit year
    - [M]M: One or two digit month
    - [D]D: One or two digit day
    
So `2019-01-10` is interpreted as January 10, 2019

The **DATETIME** format is like the date format... but with time added at the end.

## EXTRACT

Often you'll want to look at part of a date, like the year or the day. You can do this with **EXTRACT**. We'll illustrate this with a slighly different table, called `pets_with_date`.


![alt text](petsdate.PNG "pets")

The query below returns two columns, where column `Day` contains the day corresponding to each entry of the `Date` column from the `pets_with_date` table:

    SELECT Name, EXTRACT(DAY FROM Date) AS Day
    FROM `bigquery.public-data.pet_records.pets_with_date`
    
SQL is very smart about dates, ans we can ask for information beyond just extracting part of the cell. For example, this query returns one column with just the week in the year (between 1 and 53) for each date in the `Date` column:

    SELECT Name, EXTRACT(WEEK FROM Date) AS Week
    FROM `bigquery.public-data.pet_records.pets_with_date`
    
## Example: Which day of the week has the most fatal motor accidents?

Let's use the US Traffic Fatality Records database, which contains information on traffic accidents in the US where at least one person died.

We'll investigate the `accident_2015` table. Here is a view of the first few rows. 

In [1]:
# SETUP
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/levka/Downloads/KaggleSQL-79493a7efc0a.json"
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [5]:
db_ref = client.dataset("nhtsa_traffic_fatalities", project = "bigquery-public-data")
db = client.get_dataset(db_ref)
tb_ref = db.table("accident_2015")
tb = client.get_table(tb_ref)

In [6]:
client.list_rows(tb, max_results = 5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


Let's use the table to determine how the number of accidents varies with the day of the week. Since:

- the `consecutive_number` column contains a unique ID for each accident, and

- the `timestamp_of_crash` column contains the date of the accident in DATETIME format

we can:

- **EXTRACT** the day of the week (as `day_of_week` in the query below) from the `timestamp_of_crash` column, and

- **GROUP BY** the day of the week, before we **COUNT** the `consecutive_number` column to determine the number of accidents for each day of the week

Then we sort the table with an **ORDER BY** clause, so the days with the most accidents are returned first.

In [16]:
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [17]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)

accidents_by_day = query_job.to_dataframe()

accidents_by_day

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


# Exercise

The World Bank has made tons of interesting education data available through BigQuery. Run the following cell to see the first few rows of the `international_education` table from the `world_bank_intl_education` dataset.

In [6]:
client = bigquery.Client()
db_ref = client.dataset("world_bank_intl_education", project = "bigquery-public-data")
db = client.get_dataset(db_ref)
tb_ref = db.table("international_education")
tb = client.get_table(tb_ref)
client.list_rows(tb, max_results = 5).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Lebanon,LBN,Official entrance age to pre-primary education...,UIS.THAGE.0,3.000000e+00,2016
1,Lebanon,LBN,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,6.500000e+00,2016
2,Liberia,LBR,Population of the official age for secondary e...,SP.SEC.TOTL.FE.IN,3.082780e+05,2016
3,Sub-Saharan Africa (excluding high income),SSA,"Population, total",SP.POP.TOTL,1.033011e+09,2016
4,Upper middle income,UMC,"Population, ages 0-14 (% of total)",SP.POP.0014.TO.ZS,2.059912e+01,2016


## Prolific commenters

Government expenditure on education

Which countries spend the largest fraction of GDP on education?  

To answer this question, consider only the rows in the dataset corresponding to indicator code `SE.XPD.TOTL.GD.ZS`, and write a query that returns the average value in the `value` column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In [10]:
query = """
        SELECT country_name, AVG(value) AS avg_ed_spending_pct 
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE year >= 2010 AND year <= 2017 AND indicator_code = "SE.XPD.TOTL.GD.ZS" 
        GROUP BY country_name
        ORDER BY avg_ed_spending_pct DESC
        """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_results = query_job.to_dataframe()

query_results.head()

,country_name,avg_ed_spending_pct
0,Cuba,12.837270
1,"Micronesia, Fed. Sts.",12.467750
2,Solomon Islands,10.001080
3,Moldova,8.372153
4,Namibia,8.349610


## Identify interesting codes to explore

The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

In [12]:
query = """
        SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE year = 2016
        GROUP BY indicator_code, indicator_name
        HAVING num_rows >= 175
        ORDER BY num_rows DESC
        """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_results = query_job.to_dataframe()

query_results.head()

,indicator_code,indicator_name,num_rows
0,SP.POP.TOTL,"Population, total",232
1,SP.POP.GROW,Population growth (annual %),232
2,IT.NET.USER.P2,Internet users (per 100 people),223
3,SP.POP.0014.TO.ZS,"Population, ages 0-14 (% of total)",213
4,SP.POP.1564.MA.IN,"Population, ages 15-64, male",213
